<p> <center> <a href="../Start_here.ipynb">Home Page</a> </center> </p>

<div>
    <span style="float: left; width: 33%; text-align: left;"><a href="3.Model_deployment_with_Triton_Inference_Server.ipynb">Previous Notebook</a></span>
    <span style="float: left; width: 34%; text-align: center;">
        <a href="1.Data_labeling_and_preprocessing.ipynb">1</a>
        <a href="2.Object_detection_using_TAO_YOLOv4.ipynb">2</a>
        <a href="3.Model_deployment_with_Triton_Inference_Server.ipynb">3</a>
        <a >4</a>
        <a href="5.Measure_object_size_using_OpenCV.ipynb">5</a>
        <a href="6.Challenge_DeepStream.ipynb">6</a>
        <a href="7.Challenge_Triton.ipynb">7</a>
    </span>
    <span style="float: left; width: 33%; text-align: right;"><a href="5.Measure_object_size_using_OpenCV.ipynb">Next Notebook</a></span>
</div>

# Model deployment with DeepStream

***

**The goal of this notebook is to make you understand how to:**

- Deploy a TAO Toolkit model to DeepStream using Python bindings
- Find objects in a video stream, annotate them with bounding boxes, and output the annotated stream along with a count of the objects found
- Potentially scale up your application and build a multi-stream, multi-DNN pipeline

**Contents of this notebook:**

- [Introduction to DeepStream](#Introduction-to-DeepStream)
    - [Seamless integration with TAO Toolkit](#Seamless-integration-with-TAO-Toolkit)
    - [Performance and scalability](#Performance-and-scalability)
- [Overview of DeepStream SDK](#Overview-of-DeepStream-SDK)
- [GStreamer foundations](#GStreamer-foundations)
    - [Elements](#Elements)
    - [Pipelines](#Pipelines)
    - [Pads](#Pads)
    - [Caps](#Caps)
    - [Buffers](#Buffers)
    - [Plugin based architecture](#Plugin-based-architecture)
- [Getting started with DeepStream pipeline](#Getting-started-with-DeepStream-pipeline)
- [NVIDIA DeepStream plugins](#NVIDIA-DeepStream-plugins)
    - [Nvinfer](#Nvinfer)
    - [Nvtracker](#Nvtracker)
    - [Nvvidconv](#Nvvidconv)
    - [Nvosd](#Nvosd)
- [Building the pipeline](#Building-the-pipeline)
- [Understanding the configuration files](#Understanding-the-configuration-files)
- [Working with the metadata](#Working-with-the-metadata)
- [Run the pipeline](#Run-the-pipeline)
- [Scale up to multi-stream, multi-DNN pipelines](#Scale-up-to-multi-stream,-multi-DNN-pipelines)

## Introduction to DeepStream

In this notebook, you will be introduced to DeepStream, its workflow, and the underlying principles on which it is based.

![Workflow](images/ds_workflow.png)
<div style="font-size:11px">Source: https://developer.nvidia.com/deepstream-sdk</div><br>

DeepStream is a powerful streaming analytic toolkit to rapidly develop and deploy AI-powered applications and services. Some popular use cases include: automating industrial processes, robotics, optical inspection, managing logistics, and retail analytics.

DeepStream simplifies building Intelligent Video Analytics (IVA) applications by separating them into components managed and built by the user and components efficiently handled behind the scenes for us.

As developers, we build components to manage important business tasks like:
- Selecting the kind and number of video streams we want to analyze
- Choosing the type of analysis we want to do on the video
- Handling and interacting with the results of our analysis
    
We don't need to build components to manage difficult in-between tasks like:
- Efficiently leverage the GPU for accelerated processing and inference
- Efficiently process data from multiple video streams at once
- Keeping track of metadata associated with each frame of video from multiple sources
- Optimizing our pipeline for maximum data throughput
- Optimizing our neural networks for high-speed inference

Having **DeepStream SDK** solving these tasks for us, we can focus more on the valuable parts of the project related to goal and impact. 

### Seamless integration with TAO Toolkit

DeepStream offers turnkey integration of models trained with **TAO Toolkit**, speeding up overall development and deployment efforts and unlocking greater real-time performance by building an end-to-end vision AI pipeline together. With native integration to NVIDIA [Triton Inference Server](https://developer.nvidia.com/nvidia-triton-inference-server), we can also deploy models in native frameworks such as **PyTorch** and **TensorFlow** for inference. 

In this notebook, we will pick up exactly where we left off in the previous one and visualize our TAO model at work on a video stream.

<img src="images/tao_deepstream.jpeg" width="720">
<div style="width:720px; font-size:11px">Source: https://docs.nvidia.com/tao/tao-toolkit</div><br>

### Performance and scalability

DeepStream offers exceptional throughput for a wide variety of object detection, image classification and instance segmentation based AI models. The DeepStream SDK allows streaming video from multiple sources and simultaneous optimization of video decode/encode, image scaling and conversion and edge-to-cloud connectivity for complete end-to-end performance optimization.

DeepStream provides also scalability at different levels of the system hierarchy. For example: 
- DeepStream SDK 3.0 supports processing a higher number of concurrent streams, in addition to utilizing multiple GPUs upon availability
- DeepStream SDK 4.0 delivers a unified code base for all NVIDIA GPUs and quick integration with IoT services
- DeepStream SDK 5.0 adds support for NVIDIA Triton Inference Server, Python bindings, and Instance segmentation with Mask R-CNN
- DeepStream SDK 6.0 integrates NVIDIA TAO Toolkit models and introduces Graph Composer, new sample applications, and new plugins

Furthermore, DeepStream containers provide flexibility to the deployment phase and now come with libraries like CUDA, cuDNN and TensorRT already installed inside them along with the source code for reference applications. In a typical scenario, you build, execute and debug a DeepStream application within the DeepStream container. These containers are available at `ngc.nvidia.com`.

## Overview of DeepStream SDK

DeepStream SDK consists of a set of building blocks that bridge the gap between low-level APIs (such as TensorRT, Video Codec SDK) and the user application that takes streaming data input from USB/CSI camera, video from file or streams over RTSP, and uses AI and computer vision to generate insights from pixels for better understanding of the environment. DeepStream supports application development in C/C++ and in Python through the Python bindings.
DeepStream builds on top of several NVIDIA libraries from the CUDA-X stack such as `CUDA`, `TensorRT`, `NVIDIA® Triton™ Inference server` and `multimedia libraries`. DeepStream abstracts these libraries in DeepStream plugins, making it easy for developers to build video analytic pipelines without having to learn all the individual libraries. For more details visit [here](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_Overview.html).

<img src="images/deepstream_overview.jpg" width="720">
<div style="font-size:11px">Source: https://developer.nvidia.com/deepstream-sdk</div><br>

The DeepStream Python application uses the `Gst-Python` API action to construct the pipeline and use probe functions to access data at various points in the pipeline. [Click here](https://github.com/NVIDIA-AI-IOT/deepstream_python_apps/tree/master/bindings) to see the steps and prerequisites to build the custom Python binding wheel.

<img src="images/deepstream_python_bindings.png" width="720">
<div style="font-size:11px">Source: https://docs.nvidia.com/metropolis/deepstream/dev-guide</div><br>

Below, you can see a short list of the new capabilities provided by DeepStream:
- Allowing addition and removal of video stream dynamically and during the pipeline execution, in addition to frame rate and resolution adjustments
- Extending the video processing capabilities by supporting custom layers, and user-defined parsing of detector outputs
- Providing Support for a 360-degree camera using GPU-accelerated dewarping libraries
- Augmenting the meta-data with application-specific, user-defined insights
- Providing pruned and efficient inference models
- Getting detailed performance analysis with the NVIDIA Nsight Systems profiler tool

DeepStream SDK is based on the **GStreamer multimedia framework** and provides a pipeline of GPU accelerated plugins as shown below. 

The SDK facilitates the application implementation procedure by providing plugins for video inputs, video decoding, image preprocessing, TensorRT-based inference, object tracking and display, and many more. You can use these capabilities to assemble flexible, multi-GPU, multi-stream video analytics applications.

<img src="images/ds_overview.png" width="720">
<div style="width:720px; font-size:11px">Source: https://docs.nvidia.com/metropolis/deepstream/dev-guide</div><br>

## GStreamer foundations

The DeepStream SDK is based on the open source [GStreamer multimedia framework](https://gstreamer.freedesktop.org/). There are a few key concepts in GStreamer that we need to touch on before getting started. These include Elements, Pipelines, Pads, Caps, and Buffers. We will be describing them at a high level, but encourage those who are interested in the details to read the [GStreamer Basics](https://gstreamer.freedesktop.org/documentation/?gi-language=python) to learn more.

### Elements 

Elements are the core building block with which we make pipelines. Every process in-between the source (i.e. input of the pipeline, e.g. camera and video files) and sink elements (e.g. screen display) is passed through elements. Video decoding and encoding, neural network inference, and displaying text on top of video streams are examples of an element. DeepStream allows us to instantiate elements and weave them into pipelines.

### Pipelines 

All elements in GStreamer must typically be contained inside a pipeline before they can be used because it takes care of some clocking and messaging functions. A pipeline is a particular type of bin, which is the element used to contain other elements. Therefore all methods which apply to bins also apply to pipelines. We need to add the elements to the pipeline and then link them. This linking must be established following the data flow (this is, from source elements to sink elements).

![pipeline](images/pipeline.png)

### Pads

Pads are the interfaces between elements. When data flows from one element to another element in a pipeline, it flows from the sink pad of one element to the source pad of another. Note that each element might have zero, one, or many source/sink elements.

![pads](images/pads.png)

### Caps

Caps (or Capabilities) are the data types that a pad is permitted to utilize or emit. Because pads can allow multiple data types, sometimes the data flow is ambiguous. Pads are negotiated in order to explicitly define the type of data that can flow through the pad. Caps streamline this process and allow elements of our pipeline with ambiguous pads to negotiate the correct data flow process. Later in this course, we will use caps to pass certain video data types (NV12, RGB) to the downstream elements in the pipeline.

### Buffers

Buffers carry the data that will pass through the pipeline. Buffers are timestamped and contain metadata such as how many elements are using it, flags, and pointers to objects in memory. When we write application code, we rely on accessing data attached to the buffer.

### Plugin based architecture

DeepStream applications can be thought of as pipelines consisting of individual components (plugins). Each plugin represents a functional block like inference using TensorRT or multi-stream decode. Where applicable, plugins are accelerated using the underlying hardware to deliver maximum performance. DeepStream’s key value is in making deep learning for video easily accessible, to allow you to concentrate on quickly building and customizing efficient and scalable video analytics applications.

The plugin architecture provides functionality such as video encoding/decoding, scaling, inferencing, and more. By connecting plugins into a pipeline, we can build complex applications. Because DeepStream is built on top of GStreamer, we can inspect plugins using `gst-inspect-1.0`.

In [ ]:
# Inspect the nvinfer plugin
!gst-inspect-1.0 nvinfer

## Getting started with DeepStream pipeline

In this notebook, you will get started with DeepStream Python bindings and get an idea of the workflow to build a 6-class object detection pipeline with a tracker assigning unique IDs to objects. The 6-class object detection pipeline has a structure similar to the one shown in the illustration below, but the second stage classifiers (highlighted in gray) are not implemented in this case.

<img src="images/test2.png" width="1080">

We notice there are multiple DeepStream plugins used in the pipeline, let's have a look and try to understand them.

## NVIDIA DeepStream plugins

### Nvinfer

The nvinfer plugin provides TensorRT-based inference for detection and tracking. The low-level library (libnvds_infer) operates on any of RGB, BGR, or GRAY data with dimension of Network Height and Network Width. The plugin accepts NV12/RGBA data from upstream components like the decoder, muxer, and dewarper.
The Gst-nvinfer plugin also performs preprocessing operations like format conversion, scaling, mean subtraction, and produces final float RGB/BGR/GRAY planar data which is passed to the low-level library. The low-level library uses the TensorRT engine for inferencing. It outputs each classified object’s class and each detected object’s bounding boxes (Bboxes) after clustering.

<img src="images/nvinfer.png" width="720">
<div style="font-size:11px">Source: https://docs.nvidia.com/metropolis/deepstream/dev-guide</div><br>

### Nvtracker

This plugin allows the DS pipeline to use a low-level tracker library to track the detected objects with persistent (possibly unique) IDs over time. The plugin accepts NV12/RGBA data from the upstream component and scales (and/or converts) the input buffer to a buffer with a specific tracker width and height (`tracker-width` and `tracker-height` must be specified in the configuration file’s `[tracker]` section).

The path to the low-level tracker library is to be specified via the `ll-lib-file` configuration option in the same section. The low-level library to be used may also require its own configuration file, which can be specified via the `ll-config-file` option. The reference low-level tracker implementations support different tracking algorithms, including the Intersection-Over-Union (IOU) tracker algorithm to determine the object's unique ID, which uses the intersection of the detector’s bounding boxes between two consecutive frames to perform the association between them or assign a new target ID if no match found.

DeepStream 6.0 introduced a unified low-level tracker library named `libnvds_nvmultiobjecttracker.so`, therefore the tracker configuration file depending on your tracker choice will be the same for every algorithm.

<img src="images/nvtracker.png" width="720">
<div style="font-size:11px">Source: https://docs.nvidia.com/metropolis/deepstream/dev-guide</div><br>

### Nvvidconv 

The nvvidconv plugin performs video color format conversion, which is required to make data ready for the nvosd plugin.

<img src="images/nvvidconv.png" width="720">
<div style="font-size:11px">Source: https://docs.nvidia.com/metropolis/deepstream/dev-guide</div><br>

### Nvosd

The nvosd plugin draws bounding boxes, text, and region of interest (RoI) polygons (polygons are presented as a set of lines). The plugin accepts an RGBA buffer with attached metadata from the upstream component. It draws bounding boxes, which may be shaded depending on the configuration (e.g. width, color, and opacity) of a given bounding box. It also draws text and RoI polygons at specified locations in the frame. Text and polygon parameters are configurable through metadata.

<img src="images/nvosd.png" width="720">
<div style="font-size:11px">Source: https://docs.nvidia.com/metropolis/deepstream/dev-guide</div><br>

Now with that in mind, let's get started building the pipeline.

## Building the pipeline

In [ ]:
# Import required libraries
import sys
sys.path.append("../source_code/N4")
from bus_call import bus_call
import gi
gi.require_version('Gst', '1.0')
from gi.repository import GLib, Gst
import configparser
import pyds
import time

# Define class labels
PGIE_CLASS_ID_FRESHAPPLE = 0
PGIE_CLASS_ID_FRESHBANANA = 1
PGIE_CLASS_ID_FRESHORANGE = 2
PGIE_CLASS_ID_ROTTENAPPLE = 3
PGIE_CLASS_ID_ROTTENBANANA = 4
PGIE_CLASS_ID_ROTTENORANGE = 5

# Define input/output video files
INPUT_VIDEO_NAME  = "../source_code/N4/apples.h264" # Source: https://depositphotos.com
OUTPUT_VIDEO_NAME = "../source_code/N4/ds_out.mp4"

First, we define a function `make_elm_or_print_err()` to create our elements and report any errors if the creation fails. Elements are created using the `Gst.ElementFactory.make()` function as part of Gstreamer library.

In [ ]:
# Make element or print error and any other detail
def make_elm_or_print_err(factoryname, name, printedname, detail=""):
    print("Creating", printedname)
    elm = Gst.ElementFactory.make(factoryname, name)
    if not elm:
        sys.stderr.write("Unable to create " + printedname + " \n")
    if detail:
        sys.stderr.write(detail)
    return elm

Next, we initialize GStreamer and create an empty pipeline.

In [ ]:
# Standard GStreamer initialization
Gst.init(None)

# Create gstreamer elements
# Create Pipeline element that will form a connection of other elements
print("Creating Pipeline \n")
pipeline = Gst.Pipeline()

if not pipeline:
    sys.stderr.write(" Unable to create Pipeline \n")

Then, we create the elements that are required for our pipeline.

In [ ]:
########### Create elements required for the Pipeline ###########
# Source element for reading from the file
source = make_elm_or_print_err("filesrc", "file-source", "Source")
# Since the data format in the input file is elementary h264 stream, we need a h264parser
h264parser = make_elm_or_print_err("h264parse", "h264-parser", "h264 parse")
# Use nvdec_h264 for hardware accelerated decode on GPU
decoder = make_elm_or_print_err("nvv4l2decoder", "nvv4l2-decoder", "Nvv4l2 Decoder")
# Create nvstreammux instance to form batches from one or more sources
streammux = make_elm_or_print_err("nvstreammux", "Stream-muxer", "NvStreamMux")
# Use nvinfer to run inferencing on decoder's output, behavior of inferencing is set through config file
pgie = make_elm_or_print_err("nvinfer", "primary-inference", "pgie")
# Use nvtracker to give objects unique-ids
tracker = make_elm_or_print_err("nvtracker", "tracker", "tracker")
# Use convertor to convert from NV12 to RGBA as required by nvosd
nvvidconv = make_elm_or_print_err("nvvideoconvert", "convertor", "nvvidconv")
# Create OSD to draw on the converted RGBA buffer
nvosd = make_elm_or_print_err("nvdsosd", "onscreendisplay", "nvosd")
# Finally encode and save the osd output
queue = make_elm_or_print_err("queue", "queue", "Queue")
# Use convertor to convert from NV12 to RGBA as required by nvosd
nvvidconv2 = make_elm_or_print_err("nvvideoconvert", "convertor2", "nvvidconv2")
# Place an encoder instead of OSD to save as video file
encoder = make_elm_or_print_err("avenc_mpeg4", "encoder", "Encoder")
# Parse output from Encoder
codeparser = make_elm_or_print_err("mpeg4videoparse", "mpeg4-parser", "Code Parser")
# Create a container
container = make_elm_or_print_err("qtmux", "qtmux", "Container")
# Create Sink for storing the output
sink = make_elm_or_print_err("filesink", "filesink", "Sink")

Now that we have created the elements, we can proceed to set various properties for our pipeline.

## Understanding the configuration files

We set a `config-file-path` for our nvinfer (interference plugin) pointing to the file `pgie_yolov4_tao_config.txt`. In this configuration file, the `[property]` group configures the general behavior of the plugin and it is the only mandatory group. Additionally, the `[class-attrs-all]` group configures detection parameters for all classes while the `[class-attrs-<class-id>]` group does the same job but for a particular class specified by `<class-id>`. A list of the keys supported for `[property]` and `[class-attrs-…]` groups is available [here](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvinfer.html#gst-nvinfer-file-configuration-specifications).
    
You can take a look at the `pgie_yolov4_tao_config.txt` configuration file below and identify these parts. In this example, we use model-file `resnet18` and let nvinfer create a TensorRT engine specific to the host GPU to accelerate its inference performance.

Please make sure to set your API key in the configuration file [here](../source_code/N4/pgie_yolov4_tao_config.txt) in the field `tlt-model-key`, then save the file with `ctrl s`. Not doing so makes it impossible to decrypt the model and successfully run this notebook.

In [ ]:
print("Please replace the tlt-model-key variable with your key.")

In [ ]:
!cat ../source_code/N4/pgie_yolov4_tao_config.txt

We can also visualize the configuration file for our nvtracker (tracking plugin) named `dstest2_tracker_config.txt`. The configuration file is parsed and properties are then set for the tracker.

In [ ]:
!cat ../source_code/N4/dstest2_tracker_config.txt

In the next cell, we set the properties for the elements of our pipeline, including but not limited to the contents of the two configuration files.

In [ ]:
############ Set properties for the Elements ############
print("Playing file ", INPUT_VIDEO_NAME)
# Set Input File Name 
source.set_property("location", INPUT_VIDEO_NAME)
# Set Input Width, Height and Batch Size 
streammux.set_property("width", 1920)
streammux.set_property("height", 1080)
streammux.set_property("batch-size", 1)
# Timeout in microseconds to wait after the first buffer is available 
# to push the batch even if a complete batch is not formed.
streammux.set_property("batched-push-timeout", 4000000)
# Set Congifuration file for nvinfer 
pgie.set_property("config-file-path", "../source_code/N4/pgie_yolov4_tao_config.txt")
#Set properties of tracker from tracker_config
config = configparser.ConfigParser()
config.read("../source_code/N4/dstest2_tracker_config.txt")
config.sections()
for key in config['tracker']:
    if key == 'tracker-width' :
        tracker_width = config.getint('tracker', key)
        tracker.set_property('tracker-width', tracker_width)
    if key == 'tracker-height' :
        tracker_height = config.getint('tracker', key)
        tracker.set_property('tracker-height', tracker_height)
    if key == 'gpu-id' :
        tracker_gpu_id = config.getint('tracker', key)
        tracker.set_property('gpu_id', tracker_gpu_id)
    if key == 'll-lib-file' :
        tracker_ll_lib_file = config.get('tracker', key)
        tracker.set_property('ll-lib-file', tracker_ll_lib_file)
    if key == 'll-config-file' :
        tracker_ll_config_file = config.get('tracker', key)
        tracker.set_property('ll-config-file', tracker_ll_config_file)
    if key == 'enable-batch-process' :
        tracker_enable_batch_process = config.getint('tracker', key)
        tracker.set_property('enable_batch_process', tracker_enable_batch_process)
# Set Encoder bitrate for output video
encoder.set_property("bitrate", 2000000)
# Set Output file name and disable sync and async
sink.set_property("location", OUTPUT_VIDEO_NAME)
sink.set_property("sync", 0)
sink.set_property("async", 0)

We now link all the elements in the order we prefer and create Gstreamer bus to feed all messages through it.

In [ ]:
########## Add and Link Elements in the Pipeline ##########

print("Adding elements to Pipeline \n")

pipeline.add(source)
pipeline.add(h264parser)
pipeline.add(decoder)
pipeline.add(streammux)
pipeline.add(pgie)
pipeline.add(tracker)
pipeline.add(nvvidconv)
pipeline.add(nvosd)
pipeline.add(queue)
pipeline.add(nvvidconv2)
pipeline.add(encoder)
pipeline.add(codeparser)
pipeline.add(container)
pipeline.add(sink)

# We now  link the elements together 
# file-source -> h264-parser -> nvh264-decoder -> nvinfer -> nvvidconv ->
# queue -> nvvidconv2 -> encoder -> parser -> container -> sink -> output-file
print("Linking elements in the Pipeline \n")
source.link(h264parser)
h264parser.link(decoder)

##### Creating Sink pad and source pads and linking them together 

# Create Sinkpad to Streammux 
sinkpad = streammux.get_request_pad("sink_0")
if not sinkpad:
    sys.stderr.write(" Unable to get the sink pad of streammux \n")
# Create source pad from Decoder   
srcpad = decoder.get_static_pad("src")
if not srcpad:
    sys.stderr.write(" Unable to get source pad of decoder \n")
    
srcpad.link(sinkpad)
streammux.link(pgie)
pgie.link(tracker)
tracker.link(nvvidconv)
nvvidconv.link(nvosd)
nvosd.link(queue)
queue.link(nvvidconv2)
nvvidconv2.link(encoder)
encoder.link(codeparser)
codeparser.link(container)
container.link(sink)

Now we create an event loop and feed GStreamer bus messages to it.

In [ ]:
loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect ("message", bus_call, loop)

## Working with the metadata

Our pipeline now carries the metadata forward but does nothing with it up to this moment. As mentioned in the above pipeline diagram, we will now create a callback function to display relevant data on the frame once it is called and create a sink pad in the `nvosd` element to call the function.

In [ ]:
############## Working with the Metadata ################

def osd_sink_pad_buffer_probe(pad, info, u_data):
    
    # Intiallizing object counter with 0
    obj_counter = {
        PGIE_CLASS_ID_FRESHAPPLE:0,
        PGIE_CLASS_ID_FRESHBANANA:0,
        PGIE_CLASS_ID_FRESHORANGE:0,
        PGIE_CLASS_ID_ROTTENAPPLE:0,
        PGIE_CLASS_ID_ROTTENBANANA:0,
        PGIE_CLASS_ID_ROTTENORANGE:0
    }
    # Colors of the bounding boxes in RGBA
    obj_colors = {
        PGIE_CLASS_ID_FRESHAPPLE:(1.0, 0.0, 0.0, 0.0),
        PGIE_CLASS_ID_FRESHBANANA:(0.0, 1.0, 0.0, 0.0),
        PGIE_CLASS_ID_FRESHORANGE:(0.0, 0.0, 1.0, 0.0),
        PGIE_CLASS_ID_ROTTENAPPLE:(0.0, 1.0, 1.0, 0.0),
        PGIE_CLASS_ID_ROTTENBANANA:(1.0, 0.0, 1.0, 0.0),
        PGIE_CLASS_ID_ROTTENORANGE:(1.0, 1.0, 0.0, 0.0)
    }
    # Set frame_number & rectangles to draw as 0 
    frame_number=0
    num_rects=0
    
    gst_buffer = info.get_buffer()
    if not gst_buffer:
        print("Unable to get GstBuffer ")
        return

    # Retrieve batch metadata from the gst_buffer
    # Note that pyds.gst_buffer_get_nvds_batch_meta() expects the
    # C address of gst_buffer as input, which is obtained with hash(gst_buffer)
    batch_meta = pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame = batch_meta.frame_meta_list
    
    while l_frame is not None:
        try:
            # Note that l_frame.data needs a cast to pyds.NvDsFrameMeta
            frame_meta = pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break
        
        # Get frame number, number of rectangles to draw and object metadata
        frame_number=frame_meta.frame_num
        num_rects = frame_meta.num_obj_meta
        l_obj=frame_meta.obj_meta_list
        
        while l_obj is not None:
            try:
                # Casting l_obj.data to pyds.NvDsObjectMeta
                obj_meta=pyds.NvDsObjectMeta.cast(l_obj.data)
            except StopIteration:
                break
            # Increment object class by 1 and set box border color  
            obj_counter[obj_meta.class_id] += 1
            r, g, b, a = obj_colors[obj_meta.class_id]
            obj_meta.rect_params.border_color.set(r, g, b, a)
            try: 
                l_obj=l_obj.next
            except StopIteration:
                break
        ################## Setting Metadata Display configruation ############### 
        # Acquiring a display meta object
        display_meta=pyds.nvds_acquire_display_meta_from_pool(batch_meta)
        display_meta.num_labels = 1
        py_nvosd_text_params = display_meta.text_params[0]
        # Setting display text to be shown on screen
        py_nvosd_text_params.display_text = "Frame Number={} Number of Objects={} Freshapple_count={} Freshbanana_count={} " \
            "Freshorange_count={} Rottenapple_count={} Rottenbanana_count={} Rottenorange_count={}".format(frame_number, num_rects, 
            obj_counter[PGIE_CLASS_ID_FRESHAPPLE], obj_counter[PGIE_CLASS_ID_FRESHBANANA], obj_counter[PGIE_CLASS_ID_FRESHORANGE], 
            obj_counter[PGIE_CLASS_ID_ROTTENAPPLE], obj_counter[PGIE_CLASS_ID_ROTTENBANANA], obj_counter[PGIE_CLASS_ID_ROTTENORANGE])
        
        # Now set the offsets where the string should appear
        py_nvosd_text_params.x_offset = 10
        py_nvosd_text_params.y_offset = 12
        # Font, font-color and font-size
        py_nvosd_text_params.font_params.font_name = "Serif"
        py_nvosd_text_params.font_params.font_size = 14
        # Set(red, green, blue, alpha); Set to White
        py_nvosd_text_params.font_params.font_color.set(1.0, 1.0, 1.0, 1.0)
        # Text background color
        py_nvosd_text_params.set_bg_clr = 1
        # Set(red, green, blue, alpha); set to Black
        py_nvosd_text_params.text_bg_clr.set(0.0, 0.0, 0.0, 1.0)
        # Using pyds.get_string() to get display_text as string to print in notebook
        print(pyds.get_string(py_nvosd_text_params.display_text))
        pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)
        
        ############################################################################
        
        try:
            l_frame=l_frame.next
        except StopIteration:
            break
    return Gst.PadProbeReturn.OK

Here we add the probe to get informed of the meta data generated. We add probe to the sink pad of the osd element, since by that time, the buffer would have got all the metadata.

In [ ]:
osdsinkpad = nvosd.get_static_pad("sink")
if not osdsinkpad:
    sys.stderr.write(" Unable to get sink pad of nvosd \n")
    
osdsinkpad.add_probe(Gst.PadProbeType.BUFFER, osd_sink_pad_buffer_probe, 0)

## Run the pipeline

Now with everything defined, we can start the playback and listen to the events.

In [ ]:
# start play back and listen to events
print("Starting pipeline \n")
start_time = time.time()
pipeline.set_state(Gst.State.PLAYING)
try:
    loop.run()
except:
    pass
# cleanup
pipeline.set_state(Gst.State.NULL)
print("--- %s seconds ---" % (time.time() - start_time))

With the next cell, we convert the video profile to be compatible with Jupyter notebook.

In [ ]:
!ffmpeg -loglevel panic -y -an -i ../source_code/N4/ds_out.mp4 -vcodec libx264 -pix_fmt yuv420p -profile:v baseline -level 3 ../source_code/N4/output.mp4

In [ ]:
# Display the output
from IPython.display import HTML
HTML("""
 <video width="640" height="480" controls>
 <source src="../source_code/N4/output.mp4"
 </video>
""".format())

## Scale up to multi-stream, multi-DNN pipelines

In this notebook, we learned how to build a pipeline to run inference on a single stream with an object detection model trained with TAO Toolkit. However, DeepStream also provides tools for multi-stream and multi-stage pipeline implementation. For example, we might be interested in capturing video footage from multiple cameras simultaneously and setting up an object detection, tracking, and attribute classification pipeline.

If you are interested in building more complex pipelines like this, we encourage you to check the [DeepStream Python Apps](https://github.com/NVIDIA-AI-IOT/deepstream_python_apps) GitHub repository and go through the [DeepStream Bootcamp](https://github.com/openhackathons-org/gpubootcamp/tree/master/ai/DeepStream). If your focus is more on understanding the performance optimization cycle using profilers, then this other Bootcamp on [DeepStream Pipeline Optimization using Profiling](https://github.com/gpuhackathons-org/gpubootcamp/tree/master/ai/DeepStream_Perf_Lab) might be of your interest as well.

In this notebook, we have seen how to deploy our TAO model to DeepStream for Intelligent Video Analytics (IVA) applications. Next, we will see how to use OpenCV to estimate the size of objects and how to incorporate this into our machine learning pipeline to gather additional information. Please go to the next notebook by clicking on the `Next Notebook` button below.

***

## References

- [1] *https://github.com/NVIDIA-AI-IOT/deepstream_python_apps*
- [2] *https://github.com/NVIDIA-AI-IOT/deepstream_tao_apps*

## Licensing

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).

<br>
<div>
    <span style="float: left; width: 33%; text-align: left;"><a href="3.Model_deployment_with_Triton_Inference_Server.ipynb">Previous Notebook</a></span>
    <span style="float: left; width: 34%; text-align: center;">
        <a href="1.Data_labeling_and_preprocessing.ipynb">1</a>
        <a href="2.Object_detection_using_TAO_YOLOv4.ipynb">2</a>
        <a href="3.Model_deployment_with_Triton_Inference_Server.ipynb">3</a>
        <a >4</a>
        <a href="5.Measure_object_size_using_OpenCV.ipynb">5</a>
        <a href="6.Challenge_DeepStream.ipynb">6</a>
        <a href="7.Challenge_Triton.ipynb">7</a>
    </span>
    <span style="float: left; width: 33%; text-align: right;"><a href="5.Measure_object_size_using_OpenCV.ipynb">Next Notebook</a></span>
</div>

<br>
<p> <center> <a href="../Start_here.ipynb">Home Page</a> </center> </p>